# CSC411 Assignment 3
Let's get it.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

## P1: Reading Data

In [4]:
import os
root_dir = os.path.join(os.getcwd(), "data")
fake_data = pd.read_fwf(os.path.join(root_dir, "clean_fake.txt"), header=None)
real_data = pd.read_fwf(os.path.join(root_dir, "clean_real.txt"), header=None)

In [21]:
fake_train, fake_test = train_test_split(fake_data, test_size=0.15)
real_train, real_test = train_test_split(real_data, test_size=0.15)

fake_train, fake_validation = train_test_split(fake_train, test_size=0.15/0.85)
real_train, real_validation = train_test_split(real_train, test_size=0.15/0.85)

## P2: Naive Bayes

In [25]:
def generate_counts(data):
    '''
    data is an array of strings
    '''
    counts = {}
    for line in data:
        words = line.split(" ")
        for word in words:
            if word in counts:
                counts[word] += 1.0
            else:
                counts[word] = 1.0
    return counts

In [45]:
fake_counts = generate_counts(fake_train[0])
real_counts = generate_counts(real_train[0])

fake_vocab_size = len(fake_counts)
print fake_vocab_size
real_vocab_size = len(real_counts)
print real_vocab_size


for word in fake_counts.keys():
    if word not in real_counts:
        real_counts[word] = 0.0
for word in real_counts.keys():
    if word not in fake_counts:
        fake_counts[word] = 0.0

total_counts = {}
for word in real_counts.keys():
    total_counts[word] = real_counts[word] + fake_counts[word]
    
total_vocab_size = len(total_counts)
print total_vocab_size

num_fake = float(len(fake_train))
num_real = float(len(real_train))
num_total = num_fake + num_real

print num_fake
print num_real

P_real = num_real / num_total
P_fake = num_fake / num_total

print P_real
print P_fake



alpha = 1.0 # Smoothing prior, alpha=1.0 Laplace smoothing
fake_thetas = {word: (fake_counts[word] + alpha) / (num_fake + alpha * fake_vocab_size) for word in fake_counts.keys()}      #gives P(x | fake)
real_thetas = {word: (real_counts[word] + alpha) / (num_real + alpha * real_vocab_size) for word in real_counts.keys()}      #gives P(x | real)
total_thetas = {word: (total_counts[word] + alpha) / (num_total + alpha * total_vocab_size) for word in total_counts.keys()}  #gives P(x)

2995
2883
4746
908.0
1376.0
0.602451838879
0.397548161121


In [44]:
def make_prediction(line):
    '''
    line is a string
    '''
    words = line.split(" ")
    P_words_given_real = 1.0
    P_words_given_fake = 1.0
    for word in real_thetas.keys():
      
        if word in words:
            P_words_given_real *= real_thetas[word] 
            P_words_given_fake *= fake_thetas[word]
        else:
            P_words_given_real *= 1 - real_thetas[word]
            P_words_given_fake *= 1 - fake_thetas[word]
            
    numerator = P_words_given_real * P_real
    denominator = numerator + P_words_given_fake * P_fake
    #print(numerator)
    #print(denominator)
    return numerator / denominator

In [48]:
print(make_prediction("fcc chairman ajit pai attempted to take a pornstar out for dinner"))

0.255320390912


## Part 3

In [44]:
fake_freqs_ = [(word, freq) for word, freq in fake_freqs.items()]
sorted_fake_words = sorted(fake_freqs_, key=lambda pair: pair[1], reverse=True)

In [45]:
sorted_fake_words[:10]

[('trump', 1.0220264317180616),
 ('the', 0.3447136563876652),
 ('to', 0.29845814977973567),
 ('in', 0.183920704845815),
 ('donald', 0.17290748898678415),
 ('a', 0.1685022026431718),
 ('for', 0.1618942731277533),
 ('of', 0.15418502202643172),
 ('and', 0.14757709251101322),
 ('on', 0.12555066079295155)]

## Part 7: Decision Tree

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [17]:
fake_file = os.path.join(root_dir, "clean_fake.txt")
top_fake = CountVectorizer(os.path.join(root_dir, "clean_fake.txt"), stop_words='english', max_features=10)

In [20]:
cv.fit_transform([fake_file])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
fake_train_labels = np.zeros((fake_train.shape[0], 1))
real_train_labels = np.ones((real_train.shape[0], 1))
y_train = np.row_stack((fake_train_labels, real_train_labels))
X_train = 

In [20]:
all_depths = []
train_accuracies = []
valid_accuracies = []

In [21]:
max_depths = np.random.randint(low=1, high=100, size=10)
for depth in max_depths:
    print "Max Depth: {}".format(depth)
    all_depths.append(depth)
    
    tree_clf = DecisionTreeClassifier(max_depth=depth)
    tree_clf.fit(fake_train, real_train)
    
    train_accuracy = tree_clf.score(fake_train, real_train)
    valid_accuracy = tree_clf.score(fake_validation, real_validation)
    
    train_accuracies.append(train_accuracy)
    valid_accuracies.append(valid_accuracy)

Max Depth: 93


ValueError: could not convert string to float: comment on rachel maddow declares that a trump presidency could have radical effects on her chosen profession by anonymous